In [15]:
%matplotlib inline

import pandas as pd
import pickle
import time
import importlib
import numpy as np

import sys
sys.path.append('/home/ngr/gdrive/wearables/scripts')
import data as weardata

In [5]:
tic = time.time()
data = load_datadict()
print('Load in {}-s'.format(time.time() - tic))

Load in 361.50193190574646-s
